In [ ]:
# arXiv:1608.03981v1 [cs.CV] 13 Aug 2016#
# Beyond a Gaussian Denoiser: Residual Learning of
# Deep CNN for Image Denoising
# Kai Zhang, Wangmeng Zuo, Yunjin Chen, Deyu Meng, and Lei Zhang
# 
# Author: Olaf Christ
# Email: christ_o@gmx.de
# DnCNN model

import numpy as np
#import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation, Subtract, SpatialDropout2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.constraints import max_norm
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
#from keras import backend as K

#this fixes issues present on my system
import tensorflow.keras.backend as K
from keras.backend.tensorflow_backend import set_session
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

#This fixes issues with cuDNN
config = ConfigProto()
config.gpu_options.allow_growth = True
InteractiveSession(config=config).close()
session = InteractiveSession(config=config)

#this fixes out of memory errors
import tensorflow as tf
from tensorflow.keras import backend as K 
import gc
K.clear_session()
gc.collect()

class DnCNN(object):
    def __init__(self,
                 useDropout = False, 
                 activation = 'relu',
                 dropout_rate = 0.2,
                 depth = 5,
                 kernel_size = (3, 3),
                 strides=(1, 1),
                 padding = 'same',
                 data_format="channels_last",
                 dilation_rate=(1, 1),
                 kernel_initializer='he_normal'):
                
                self.useDropout = useDropout
                self.depth = depth
                self.kernel_size = kernel_size
                self.strides = strides
                self.dropout_rate = dropout_rate
                self.activation = activation
                self.padding = padding
                self.data_format = data_format
                self.dilation_rate = dilation_rate

    def build(self, anInput):
        
                x = Conv2D(filters=64, kernel_size=self.kernel_size, strides=self.strides, padding=self.padding)(anInput)
                x = Activation(self.activation)(x)
                for i in range(self.depth):
                        x = Conv2D(filters = 64,
                               kernel_size = self.kernel_size,
                               strides= self.strides,
                               padding= self.padding,
                               data_format=self.data_format,
                               dilation_rate=self.dilation_rate)(x)
                        if self.useDropout:
                            x = SpatialDropout2D(self.dropout_rate)(x)    
                        x = BatchNormalization(axis=-1, epsilon=1e-3)(x)
                        x = Activation(self.activation)(x)   

                x = Conv2D(filters = 1,
                               kernel_size = self.kernel_size,
                               strides= self.strides,
                               padding= self.padding,
                               data_format=self.data_format,
                               dilation_rate=self.dilation_rate)(x)
                if self.useDropout:
                    x = SpatialDropout2D(self.dropout_rate)(x)
                x = Subtract()([anInput, x])# subtract noise
                model = Model(inputs=anInput, outputs=x)    
                return model            

In [ ]:
# arXiv:1608.03981v1 [cs.CV] 13 Aug 2016#
# Beyond a Gaussian Denoiser: Residual Learning of
# Deep CNN for Image Denoising
# Kai Zhang, Wangmeng Zuo, Yunjin Chen, Deyu Meng, and Lei Zhang
# 
# Author: Olaf Christ
# Email: christ_o@gmx.de
# Train the DnCNN on MNIST
# 

from tensorflow.keras.datasets import mnist

# the code to load the MNIST dataset is adapted from
# https://www.machinecurve.com/index.php/2019/12/20/building-an-image-denoiser-with-a-keras-autoencoder-neural-network/
img_width, img_height = 28, 28
noise_factor = 0.75

# Load MNIST dataset
(input_train, target_train), (input_test, target_test) = mnist.load_data()

input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 1)
input_test = input_test.reshape(input_test.shape[0], img_width, img_height, 1)
input_shape = (img_width, img_height, 1)

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Add noise
pure = input_train
pure_test = input_test
noise = np.random.normal(0, 1, pure.shape)
noise_test = np.random.normal(0, 1, pure_test.shape)
noisy_input = pure + noise_factor * noise
noisy_input_test = pure_test + noise_factor * noise_test
batch_size = 8

# Network parameters
INPUT_SHAPE = (img_width, img_height, 1)
inputs = Input(shape=INPUT_SHAPE, name='encoder_input')
aDnCNN = DnCNN()
denoisingCNNModel = aDnCNN.build(anInput = inputs)
denoisingCNNModel.summary()
denoisingCNNModel.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

#stop training if val_accuracy doesn't improve for 10 epochs.
earlystop_callback = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# Train the autoencoder
history = denoisingCNNModel.fit(noisy_input, pure,
                validation_split=0.2,
                epochs=15,
                batch_size=batch_size,
                callbacks=[earlystop_callback])

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

number_of_visualizations = 4

# Generate denoised images
samples = noisy_input_test[:number_of_visualizations]
targets = target_test[:number_of_visualizations]
denoised_images = denoisingCNNModel.predict(samples)

# Plot denoised images
for i in range(0, number_of_visualizations):
  # Get the sample and the reconstruction
      noisy_image = noisy_input_test[i][:, :, 0]
      pure_image  = pure_test[i][:, :, 0]
      denoised_image = denoised_images[i][:, :, 0]
      input_class = targets[i]
      # Matplotlib preparations
      fig, axes = plt.subplots(1, 3)
      fig.set_size_inches(8, 3.5)
      # Plot sample and reconstruciton
      axes[0].imshow(noisy_image)
      axes[0].set_title('Noisy image')
      axes[1].imshow(pure_image)
      axes[1].set_title('Pure image')
      axes[2].imshow(denoised_image)
      axes[2].set_title('Denoised image')
      fig.suptitle(f'MNIST target = {input_class}')
plt.show()

In [ ]:
# arXiv:1608.03981v1 [cs.CV] 13 Aug 2016#
# Beyond a Gaussian Denoiser: Residual Learning of
# Deep CNN for Image Denoising
# Kai Zhang, Wangmeng Zuo, Yunjin Chen, Deyu Meng, and Lei Zhang
# 
# Author: Olaf Christ
# Email: christ_o@gmx.de
# Copyright (C) Olaf Christ 2020, All rights reserved
#
# Train the DnCNN on 256x256 patches randomly extracted from a few pictures
# 

import os
import glob
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.feature_extraction import image
from tensorflow.keras.preprocessing import image as tf_Image
from PIL import Image as pil_image

def trainingDatagenerator(y_, batch_size=8):
    
    # y_ are the clean images
    indices = list(range(y_.shape[0]))
    while(True):
        np.random.shuffle(indices)
        for i in range(0, len(indices), batch_size):
            batch_y = y_[indices[i:i+batch_size]]
            noise =  np.random.normal(0, 1, batch_y.shape)
            batch_x = batch_y + noise  #add some noise to the clean batch
            yield batch_x, batch_y
            
#take a few images and randomly sample patches from those images
TRAIN_IMAGES = glob.glob('./monkey_images/fullImagesGrayscale/*.jpg')
training_data_clean = np.zeros((4000, 256, 256, 1))

#prepare clean data 
cnt = 0
for i, fig in enumerate(TRAIN_IMAGES):
    one_image = tf_Image.load_img(fig, color_mode='grayscale')
    custom_image = tf_Image.img_to_array(one_image).astype('float32')
    random_patches = image.extract_patches_2d(custom_image, (256, 256), max_patches = 1000)
    for x in range(1000):
        img = pil_image.fromarray(random_patches[x])
        data = tf_Image.img_to_array(img).astype('float32')
        data = data/256.0
        training_data_clean[x+cnt] = data
    cnt = cnt + 1000   

# Network parameters
INPUT_SHAPE = (256, 256, 1)
inputs = Input(shape=INPUT_SHAPE, name='encoder_input')
aDnCNN = DnCNN()
denoisingCNNModel = aDnCNN.build(anInput = inputs)
denoisingCNNModel.summary()
denoisingCNNModel.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
history = denoisingCNNModel.fit(trainingDatagenerator(training_data_clean), steps_per_epoch=len(training_data_clean)//8)

In [ ]:
# arXiv:1608.03981v1 [cs.CV] 13 Aug 2016#
# Beyond a Gaussian Denoiser: Residual Learning of
# Deep CNN for Image Denoising
# Kai Zhang, Wangmeng Zuo, Yunjin Chen, Deyu Meng, and Lei Zhang
# 
# Author: Olaf Christ
# Email: christ_o@gmx.de
# Copyright (C) Olaf Christ 2020, All rights reserved
#
# Generate small patches from images at different scales
# 

import os
import glob
import numpy as np 
import cv2

def savePatches(patcharray, fileName = './DnCNN/numpyData/cleanTrain400Patches.npy'):
    numpyPatcharray = np.array(patcharray, dtype='uint8')
    np.save(fileName, numpyPatcharray)
    
def augmentData(img, mode=0):
    if mode == 0:
        return img
    elif mode == 1:
        return np.flipud(img)
    elif mode == 2:
        return np.rot90(img, np.random.randint(1,5))
    elif mode == 3:
        return np.flipud(np.rot90(img, np.random.randint(1,5)))
    elif mode == 4:
        return np.fliplr(img)
    elif mode == 5:
        return np.fliplr(np.rot90(img, np.random.randint(1,5)))

def makePatches(file_name):
    # I am aware that all this can also be done using skimage and sklearn
    patchSize, stride = 64, 10
    img = cv2.imread(file_name, 0)
    h, w = img.shape
    scales = [1, 0.9, 0.8, 0.7, 0.6, 0.5]
    patches = []
    
    for s in scales:
        print("scale:", s)
        resized_h, resized_w = int(h*s),int(w*s)
        if True: # if h == 256 and w == 256: # this is to filter out any images not 256x256 pixels.
                resizedImage = cv2.resize(img, (resized_h,resized_w), interpolation=cv2.INTER_CUBIC)
                for i in range(0, resized_h-patchSize+1, stride):
                    for j in range(0, resized_w-patchSize+1, stride):
                        x = resizedImage[i:i+patchSize, j:j+patchSize]

                        for k in range(0, 1):
                            augmentedPatch = augmentData(x, mode=np.random.randint(0,5))
                            patches.append(augmentedPatch)
    
    return patches

TRAIN_IMAGES = glob.glob('./DnCNN/testsets/Train400/*.png')
patchArray = []

for i, fileName in enumerate(TRAIN_IMAGES):
    print(fileName)
    patches = makePatches(fileName)
    patchArray = patchArray + patches
    
savePatches(patchArray)


In [ ]:
# arXiv:1608.03981v1 [cs.CV] 13 Aug 2016#
# Beyond a Gaussian Denoiser: Residual Learning of
# Deep CNN for Image Denoising
# Kai Zhang, Wangmeng Zuo, Yunjin Chen, Deyu Meng, and Lei Zhang
# 
# Author: Olaf Christ
# Email: christ_o@gmx.de
#
# Train the network on the patches and denoise a picture.

from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from PIL import Image as pil_image

# From tf.keras documentation
# This function keeps the learning rate at 0.001 for the first ten epochs
# and decreases it exponentially after that.
def scheduler(epoch):
    if epoch < 5:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 * (10 - epoch))

lLrScheduler = LearningRateScheduler(scheduler)    
    
def trainingDatagenerator(y_, batch_size=8):
    
    # y_ are the clean images
    indices = list(range(y_.shape[0]))
    while(True):
        np.random.shuffle(indices)
        for i in range(0, len(indices), batch_size):
            batch_y = y_[indices[i:i+batch_size]]
            noise =  np.random.normal(0, 0.1, batch_y.shape)
            batch_x = batch_y + noise  #add some noise to the clean batch
            yield batch_x, batch_y
            
def loadPatches(fileName = './DnCNN/numpyData/cleanTrain400Patches.npy'):
    patcharray = np.load(fileName)
    patcharray = patcharray.reshape((patcharray.shape[0],patcharray.shape[1],patcharray.shape[2],1))
    patcharray = patcharray.astype('float32')/255.0
    return patcharray            

#aModelCheckpoint = ModelCheckpoint('./DnCNN', monitor='val_loss', verbose=0, save_best_only=False,
#    save_weights_only=False, mode='auto', save_freq='epoch')

patchData = loadPatches()


INPUT_SHAPE = (None,None,1)
inputs = Input(shape=INPUT_SHAPE, name='encoder_input')
aDnCNN = DnCNN(depth=5)
denoisingCNNModel = aDnCNN.build(anInput = inputs)
denoisingCNNModel.summary()
denoisingCNNModel.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
batchSize = 128
history = denoisingCNNModel.fit(trainingDatagenerator(patchData, batch_size = batchSize), callbacks=[lLrScheduler],
                                steps_per_epoch=len(patchData)//batchSize, epochs = 5)

# save the final model
denoisingCNNModel.save('./DnCNN/DnCNN_Model/DnCNN_Model.h5') 

#test the network 
cleanImageArray = np.array(pil_image.open('./DnCNN/testsets/Set12/01.png'), dtype='float32') / 255.0
noisyImageArray = cleanImageArray + np.random.normal(0, 0.1, cleanImageArray.shape)
noisyImageArray = noisyImageArray.astype('float32')
y_predict = denoisingCNNModel.predict(noisyImageArray.reshape(1, noisyImageArray.shape[0], noisyImageArray.shape[1], 1))
outputImageArray = y_predict.reshape(cleanImageArray.shape)
outputImageArray = np.clip(outputImageArray, 0, 1)

noiseImage = pil_image.fromarray((noisyImageArray*255).astype('uint8'))
cleanImage = pil_image.fromarray((cleanImageArray*255).astype('uint8'))
outputImage = pil_image.fromarray((outputImageArray*255).astype('uint8'))
cleanImage.show()
outputImage.show()
noiseImage.show()

In [17]:
# arXiv:1608.03981v1 [cs.CV] 13 Aug 2016#
# Beyond a Gaussian Denoiser: Residual Learning of
# Deep CNN for Image Denoising
# Kai Zhang, Wangmeng Zuo, Yunjin Chen, Deyu Meng, and Lei Zhang
# 
# Author: Olaf Christ
# Email: christ_o@gmx.de
#
# Use TFLite to denoise images


import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image as pil_image
import numpy as np

#DnCNN_Model_cleanTrain400Patches_256x256 has been created with input shape 256,256,1 to avoid a conversion error
model = model = load_model('./DnCNN/DnCNN_Model/DnCNN_Model_cleanTrain400Patches_256x256.h5')
#convert to tflite and save to disk
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("./DnCNN/DnCNN_Model/DnCNN_Model_cleanTrain400Patches_256x25.tflite", "wb").write(tflite_model)

interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the TensorFlow Lite model on random input data.
input_shape = input_details[0]['shape']
#test the network 
cleanImageArray = np.array(pil_image.open('./DnCNN/testsets/Set12/01.png'), dtype='float32') / 255.0
noisyImageArray = cleanImageArray + np.random.normal(0, 0.1, cleanImageArray.shape)
noisyImageArray = noisyImageArray.astype('float32')

interpreter.set_tensor(input_details[0]['index'], noisyImageArray.reshape(1, noisyImageArray.shape[0], noisyImageArray.shape[1], 1))
interpreter.invoke()

tflite_results = interpreter.get_tensor(output_details[0]['index'])
outputImageArray=tflite_results[0].reshape(256,256)
outputImage = pil_image.fromarray((outputImageArray*255).astype('uint8'))
noiseImage = pil_image.fromarray((noisyImageArray*255).astype('uint8'))
cleanImage = pil_image.fromarray((cleanImageArray*255).astype('uint8'))

cleanImage.show()
outputImage.show()
noiseImage.show()


In [ ]:
# This is an implementation of the 2016 paper 
# "Image Restoration Using Convolutional Auto-encoders with Symmetric Skip Connections"
# https://arxiv.org/pdf/1606.08921v3.pdf
# Author: Olaf Christ
# Email: christ_o@gmx.de

from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten, SpatialDropout2D, ReLU
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.constraints import max_norm
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

#this fixes issues present on my system
import tensorflow.keras.backend as K
from keras.backend.tensorflow_backend import set_session
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

#This fixes issues with cuDNN
config = ConfigProto()
config.gpu_options.allow_growth = True
InteractiveSession(config=config).close()
session = InteractiveSession(config=config)

#this fixes out of memory errors
import tensorflow as tf
from tensorflow.keras import backend as K 
import gc
K.clear_session()
gc.collect()


class AutoEncoder(object):
    def __init__(self,
                 activation = 'relu',
                 dropout_rate = 0.2,
                 filterSize = 32,
                 skipIndices = set([9,8]),
                 numberOfLayers = 10,
                 encoderFilters = [64, 32, 16, 8],
                 decoderFilters = [64, 32, 16, 8],
                 kernel_size = (3, 3),
                 strides=(1, 1),
                 padding = 'same',
                 data_format="channels_last",
                 dilation_rate=(1, 1),
                 use_bias=False,
                 kernel_initializer='he_normal',
                 bias_initializer=None,
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=max_norm(2.0),
                 bias_constraint=None):
        
        self.skipIndices = skipIndices
        self.numberOfLayers = numberOfLayers
        self.filtersize = filterSize
        self.encoderFilters = encoderFilters
        self.decoderFilters = decoderFilters
        self.kernel_size = kernel_size
        self.strides = strides
        self.dropout_rate = dropout_rate
        self.activation = activation
        self.padding = padding
        self.data_format = data_format
        self.dilation_rate = dilation_rate
        self.use_bias = use_bias
        self.bias_initializer = bias_initializer
        self.kernel_initializer = kernel_initializer
        self.kernel_regularizer = kernel_regularizer
        self.bias_regularizer = bias_regularizer
        self.activity_regularizer = activity_regularizer
        self.kernel_constraint = kernel_constraint
        self.bias_constraint = bias_constraint
        self.conv_id=1
   
    def _buildAutoencoderWithSkipConnecions(self, inputs):
                convLayerList = list()
                x = inputs
                id = 0
                for layers in range(self.numberOfLayers):
                    x = self._makeConvBlock(self.filtersize, x, f'conv{id}')
                    x = SpatialDropout2D(self.dropout_rate)(x)
                    convLayerList.append(x)
                    id = id + 1
                y = x   
                id = 0

                for layers in range(self.numberOfLayers):
                    y = self._makeDeconvBlock(self.filtersize, y, f'deconv{id}')
                    y = SpatialDropout2D(self.dropout_rate)(y)
                    if id in self.skipIndices:
                        layer1 = convLayerList[self.numberOfLayers-id*2-2]
                        layer2 = y
                        y = self._addSkipConnectionLayer(layer1,layer2,f'skipId{id}')
                    id = id + 1
                y = Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')(y)
                return y
           
       
    def _getLayerList(model):
        return [layer for layer in model.layers], [layer.output for layer in model.layers]
   
    def _makeConvBlock(self, filters, layer, name):
        _layer = layer
        
        _layer = Conv2D(filters = filters,
                   kernel_size = self.kernel_size,
                   strides= self.strides,
                   padding= self.padding,
                   data_format=self.data_format,
                   dilation_rate=self.dilation_rate,
                   activation=self.activation,
                   use_bias=self.use_bias,
                   kernel_initializer=self.kernel_initializer,
                   bias_initializer=self.bias_initializer,
                   kernel_regularizer=self.kernel_regularizer,
                   bias_regularizer=self.bias_regularizer,
                   activity_regularizer=self.activity_regularizer,
                   kernel_constraint=self.kernel_constraint,
                   bias_constraint=self.bias_constraint,
                   name=name)(_layer)
        return _layer
       
    def _makeDeconvBlock(self, filters, layer, name):
        _layer = layer
        _layer = Conv2DTranspose(filters = filters,
                   kernel_size = self.kernel_size,
                   strides= self.strides,
                   padding= self.padding,
                   data_format=self.data_format,
                   dilation_rate=self.dilation_rate,
                   activation=self.activation,
                   use_bias=self.use_bias,
                   kernel_initializer=self.kernel_initializer,
                   bias_initializer=self.bias_initializer,
                   kernel_regularizer=self.kernel_regularizer,
                   bias_regularizer=self.bias_regularizer,
                   activity_regularizer=self.activity_regularizer,
                   kernel_constraint=self.kernel_constraint,
                   bias_constraint=self.bias_constraint,
                   name=name)(_layer)
        return _layer
   
    def _buildDeconv(self,inputs):
        x = inputs
        id = 1
        for filters in self.decoderFilters[::-1]:
            x = Conv2DTranspose(filters = filters,
                   kernel_size = self.kernel_size,
                   strides= self.strides,
                   padding= self.padding,
                   data_format=self.data_format,
                   dilation_rate=self.dilation_rate,
                   activation=self.activation,
                   use_bias=self.use_bias,
                   kernel_initializer=self.kernel_initializer,
                   bias_initializer=self.bias_initializer,
                   kernel_regularizer=self.kernel_regularizer,
                   bias_regularizer=self.bias_regularizer,
                   activity_regularizer=self.activity_regularizer,
                   kernel_constraint=self.kernel_constraint,
                   bias_constraint=self.bias_constraint,
                   name=f'deconv{id}')(x)
            id = id + 1
        x = SpatialDropout2D(self.dropout_rate)(x)
        return x   
       
    def _buildConv(self, inputs):
        x = inputs
        id = 1
        for filters in self.encoderFilters:
            x = Conv2D(filters = filters,
                   kernel_size = self.kernel_size,
                   strides= self.strides,
                   padding= self.padding,
                   data_format=self.data_format,
                   dilation_rate=self.dilation_rate,
                   activation=self.activation,
                   use_bias=self.use_bias,
                   kernel_initializer=self.kernel_initializer,
                   bias_initializer=self.bias_initializer,
                   kernel_regularizer=self.kernel_regularizer,
                   bias_regularizer=self.bias_regularizer,
                   activity_regularizer=self.activity_regularizer,
                   kernel_constraint=self.kernel_constraint,
                   bias_constraint=self.bias_constraint,
               name=f'conv{id}')(x)
            id = id + 1
        x = SpatialDropout2D(self.dropout_rate)(x)  
        return x
   
    def makeAutoencoderModelWithSkipConnecions(self,inputs):
        encoded = self._buildAutoencoderWithSkipConnecions(inputs=inputs)
        return Model(inputs=inputs, outputs=encoded, name="AutoencoderModel")
   
    def makeDecoderModel(self,inputs):
        decoded = self._buildDeconv(inputs=inputs)
        return Model(inputs=inputs, outputs=decoded, name="DecoderModel")
   
    def makeEncoderModel(self,inputs):
        encoded = self._buildConv(inputs=inputs)
        return Model(inputs=inputs, outputs=encoded, name="EncoderModel")
   
    def _addSkipConnectionLayer(self,input1, input2, name):
        x = tf.keras.layers.add([input1, input2])
        return ReLU(name=name)(x)    


In [ ]:
# This is an implementation of the 2016 paper 
# "Image Restoration Using Convolutional Auto-encoders with Symmetric Skip Connections"
# https://arxiv.org/pdf/1606.08921v3.pdf
# Author: Olaf Christ
# Email: christ_o@gmx.de
# Copyright (C) Olaf Christ 2020, All rights reserved
#
# Train the autoencoder on MNIST
#

from tensorflow.keras.datasets import mnist

# the code to load the MNIST dataset is adapted from
# https://www.machinecurve.com/index.php/2019/12/20/building-an-image-denoiser-with-a-keras-autoencoder-neural-network/
img_width, img_height = 28, 28
noise_factor = 0.75

# Load MNIST dataset
(input_train, target_train), (input_test, target_test) = mnist.load_data()

input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 1)
input_test = input_test.reshape(input_test.shape[0], img_width, img_height, 1)
input_shape = (img_width, img_height, 1)

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Add noise
pure = input_train
pure_test = input_test
noise = np.random.normal(0, 1, pure.shape)
noise_test = np.random.normal(0, 1, pure_test.shape)
noisy_input = pure + noise_factor * noise
noisy_input_test = pure_test + noise_factor * noise_test
batch_size = 64

# Network parameters
INPUT_SHAPE = (img_width, img_height, 1)
inputs = Input(shape=INPUT_SHAPE, name='encoder_input')
conv2DStackOfLayers = AutoEncoder(skipIndices = set([14]),numberOfLayers = 15)
autoEncoderModel = conv2DStackOfLayers.makeAutoencoderModelWithSkipConnecions(inputs=inputs)
autoEncoderModel.summary()
autoEncoderModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#stop training if val_accuracy doesn't improve for 10 epochs.
earlystop_callback = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# Train the autoencoder
history = autoEncoderModel.fit(noisy_input, pure,
                validation_split=0.2,
                epochs=30,
                batch_size=batch_size,
                callbacks=[earlystop_callback])


# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

number_of_visualizations = 4

# Generate denoised images
samples = noisy_input_test[:number_of_visualizations]
targets = target_test[:number_of_visualizations]
denoised_images = autoEncoderModel.predict(samples)

# Plot denoised images
for i in range(0, number_of_visualizations):
  # Get the sample and the reconstruction
      noisy_image = noisy_input_test[i][:, :, 0]
      pure_image  = pure_test[i][:, :, 0]
      denoised_image = denoised_images[i][:, :, 0]
      input_class = targets[i]
      # Matplotlib preparations
      fig, axes = plt.subplots(1, 3)
      fig.set_size_inches(8, 3.5)
      # Plot sample and reconstruciton
      axes[0].imshow(noisy_image)
      axes[0].set_title('Noisy image')
      axes[1].imshow(pure_image)
      axes[1].set_title('Pure image')
      axes[2].imshow(denoised_image)
      axes[2].set_title('Denoised image')
      fig.suptitle(f'MNIST target = {input_class}')
plt.show()



In [ ]:
# This is an implementation of the 2016 paper 
# "Image Restoration Using Convolutional Auto-encoders with Symmetric Skip Connections"
# https://arxiv.org/pdf/1606.08921v3.pdf
# Author: Olaf Christ
# Email: christ_o@gmx.de
# Copyright (C) Olaf Christ 2020, All rights reserved
#
# Train the autoencoder on 256x256 patches randomly extracted from larger images
#

%matplotlib inline
import os
import glob

import numpy as np 
import matplotlib.pyplot as plt
import cv2

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import max_norm

# This fixes Cuda issues on my installation
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
InteractiveSession(config=config).close()
session = InteractiveSession(config=config)

from tensorflow.keras import backend as K 
import gc
K.clear_session()
gc.collect()


from sklearn.feature_extraction import image
from tensorflow.keras.preprocessing import image as tf_Image
from PIL import Image as pil_image

#take a few images and randomly sample patches from those images
TRAIN_IMAGES = glob.glob('./monkey_images/fullImagesGrayscale/*.jpg')
training_data_clean = np.zeros((8000, 256, 256, 1))

#prepare clean data 
cnt = 0
for i, fig in enumerate(TRAIN_IMAGES):
    one_image = tf_Image.load_img(fig, color_mode='grayscale')
    custom_image = tf_Image.img_to_array(one_image).astype('float32')
    random_patches = image.extract_patches_2d(custom_image, (256, 256), max_patches = 2000)
    for x in range(2000):
        img = pil_image.fromarray(random_patches[x])
        data = tf_Image.img_to_array(img).astype('float32')
        data = data/256.0
        training_data_clean[x+cnt] = data
    cnt = cnt + 2000   


# Add noise
noise_factor = 0.15
pure = training_data_clean
noise = np.random.normal(0, 1, pure.shape)
noisy_input = pure + noise_factor * noise
batch_size = 8

#display clean patch
img2 = tf_Image.array_to_img(pure[0], data_format = 'channels_last')
img2.show()

#display clean noisy patch
img3 = tf_Image.array_to_img(noisy_input[0], data_format = 'channels_last')
img3.show()

# Network parameters
INPUT_SHAPE = (256, 256, 1)
inputs = Input(shape=INPUT_SHAPE, name='encoder_input')
conv2DStackOfLayers = AutoEncoder(numberOfLayers = 5, skipIndices = set())
autoEncoderModel = conv2DStackOfLayers.makeAutoencoderModelWithSkipConnecions(inputs=inputs)
autoEncoderModel.summary()
optimizer = Adam(lr=0.001)
autoEncoderModel.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])

history = autoEncoderModel.fit(noisy_input, pure, validation_split=0.2,             
                epochs=30,
                batch_size=batch_size)